In [1]:
%cd ..
import os
import time

/data/apperception/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
# disable_cache()

/home/local/apperception/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)



Using cuda:0


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [3]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [4]:
import cv2
def construct_video(frames, size=(1600, 900), base=False):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    display_video = cv2.VideoWriter(f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.avi",fourcc, 1, size)
    for frame in frames:
        img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [5]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

world = world.filter(
    (obj1.id != obj2.id) &
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
    F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 50, 180) &
    F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
    (F.min_distance(cam.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)


In [6]:
base_query_start = time.time()
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'])
print(f'optimized query time: {time.time()-base_query_start}')

YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0757-CAM_FRONT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame
None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection
None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection3D.From2DAndRoad


237it [00:00, 1122.91it/s]


None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking3D.From2DAndRoad


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 237/237 [00:00<00:00, 4894.46it/s]


None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  SegmentTrajectory.FromTracking3D
None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
pairs for obj 93: [(298.8204497922415, 682.4423743486885), (299.01599470019426, 681.809949218556), (298.84759481366257, 681.85149661344), (298.6919255951989, 681.6866275986694), (298.60381011120813, 681.4493967920732), (298.42896747734795, 681.3603075471352), (298.2936272334375, 681.2875447204339), (298.2792960009025, 680.9857008407138), (298.1676737409123, 680.9061962779896), (298.1963681137194, 680.51101

In [7]:
id_time_camId_filename_base = world.get_id_time_camId_filename(2)

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId,t1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.itemId<>t1.itemId) AND (t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND (t1.objectType LIKE 'car' OR t1.objectType LIKE 'truck') AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t1.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t1.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (EXISTS(SELECT 1
            FROM SegmentPolygon
     

In [8]:
id_time_camId_filename_base

[('scene-0757-CAM_FRONT_obj_23',
  'scene-0757-CAM_FRONT_obj_98',
  datetime.datetime(2018, 8, 30, 12, 25, 18, 112404, tzinfo=datetime.timezone.utc),
  'scene-0757-CAM_FRONT',
  'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg'),
 ('scene-0757-CAM_FRONT_obj_44',
  'scene-0757-CAM_FRONT_obj_23',
  datetime.datetime(2018, 8, 30, 12, 25, 11, 162404, tzinfo=datetime.timezone.utc),
  'scene-0757-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657111162404.jpg'),
 ('scene-0757-CAM_FRONT_obj_44',
  'scene-0757-CAM_FRONT_obj_23',
  datetime.datetime(2018, 8, 30, 12, 25, 12, 262404, tzinfo=datetime.timezone.utc),
  'scene-0757-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657112262404.jpg'),
 ('scene-0757-CAM_FRONT_obj_44',
  'scene-0757-CAM_FRONT_obj_23',
  datetime.datetime(2018, 8, 30, 12, 25, 14, 362404, tzinfo=datetime.timezone.utc),
  'scene-0757-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-08-30-1

In [9]:
base_results = set([e[4] for e in id_time_camId_filename_base])
construct_video(base_results, base=True)

In [10]:
base_results - optimized_results

NameError: name 'optimized_results' is not defined

In [6]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'], base=False)

YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0757-CAM_FRONT --------------------------------------------------------------------------------
Stage:  InView
237
None
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  DecodeFrame.ParallelDecodeFrame
None
237
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection
None
237
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

237it [00:00, 2271.50it/s]


None
237
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  DetectionEstimation
ego_speed:  1.3581198756421884


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 236/236 [00:22<00:00, 10.28it/s]


number of skipped 73
{None: 12, 'ego_exit_segment': 3, 'car_exit_segment': 22, 'meet_up': 127}
total_run_time 22.968071460723877
total_detection_time 16.16185474395752
total_generate_sample_plan_time 6.384449243545532
237
237
  filtered frames: 69.19831223628692%
..........KKKKKKKKKKKK..............................................................KKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  Tracking2D.StrongSORT
None
237
  filtered frames: 69.19831223628692%
..........KKKKKKKKKKKK..............................................................KKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  Tracking3D.From2DAndRoad


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 237/237 [00:00<00:00, 6793.58it/s]


None
237
  filtered frames: 69.19831223628692%
..........KKKKKKKKKKKK..............................................................KKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  SegmentTrajectory.FromTracking3D
None
237
  filtered frames: 69.19831223628692%
..........KKKKKKKKKKKK..............................................................KKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
pairs for obj 93: [(298.8204497922415, 682.4423743486885), (299.01599470019426, 681.809949218556), (298.60381011120813, 681.4493967920732), (298.42896747734795, 681.3603075471352), (298.2936272334375, 681.2875447204339), (298.2792960009025, 680.9857008407138), (298.1676737409123, 680.9061962779896), (297.312683057346, 679.5672499076597), (296.933810981047, 679.559164703847), (295.61

In [7]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(2)
print(f'optimized query time: {time.time()-optimized_query_start}')


execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId,t1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.itemId<>t1.itemId) AND (t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND (t1.objectType LIKE 'car' OR t1.objectType LIKE 'truck') AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t1.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t1.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (EXISTS(SELECT 1
            FROM SegmentPolygon
     

UndefinedFunction: function objectheading(tfloat, timestamp with time zone) does not exist
LINE 11:         )) AND angleBetween(facingRelative((objectHeading(t0...
                                                     ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [13]:
optimized_results = set([e[4] for e in id_time_camId_filename])
construct_video(optimized_results)

In [ ]:
investigation = [[5, 1066.0, 270.0, 28.0, 68.0], [19, 1032.0, 274.0, 27.0, 62.0], [37, 503.0, 439.0, 60.0, 51.0], [44, 615.0, 383.0, 93.0, 110.0], [73, 1072.0, 338.0, 35.0, 26.0], [82, 858.0, 308.0, 20.0, 68.0], [93, 999.0, 379.0, 493.0, 171.0], [98, 2.0, 422.0, 413.0, 160.0], [101, 568.0, 445.0, 36.0, 22.0]]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open('/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini/samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for obj in investigation:
    obj_id = obj[0]
    x,y,w,h = obj[1:]
    plt.text(x, y+15, 'obj'+str(obj_id))
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()